# Core Imports

In [1]:
# Custom Imports
from polysaccharide import general
from polysaccharide.general import optional_in_place
from polysaccharide.extratypes import ResidueSmarts

from polysaccharide.molutils import reactions
from polysaccharide.molutils.rdmol.rdtypes import *
from polysaccharide.molutils.rdmol import rdcompare, rdconvert, rdkdraw, rdcompare, rdprops, rdbond, rdlabels

from polysaccharide.polymer import monomer as monoutils
from polysaccharide.polymer.monomer import MonomerInfo
from polysaccharide.polymer.management import PolymerManager

from polysaccharide.polymer import building
import mbuild as mb

# Generic Imports
import re
from functools import partial, cached_property
from collections import defaultdict
from itertools import combinations, chain
from ast import literal_eval

# Numeric imports
import pandas as pd
import numpy as np

# File I/O
from pathlib import Path
import csv, json, openpyxl

# Typing and Subclassing
from typing import Any, Callable, ClassVar, Generator, Iterable, Optional, Union
from dataclasses import dataclass, field
from abc import ABC, abstractmethod, abstractproperty
from openmm.unit import Unit, Quantity

# Cheminformatics
from rdkit import Chem
from rdkit.Chem import rdChemReactions

from openff.toolkit import ForceField
from openff.toolkit.topology import Topology, Molecule

# Static Paths
RAW_DATA_PATH  = Path('raw_monomer_data')
PROC_DATA_PATH = Path('processed_monomer_data')
RXN_FILES_PATH = Path('rxn_smarts')
MONO_INFO_DIR  = Path('monomer_files')

/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/timber/miniconda3/envs/openff-dev-updated/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Wa

# File and chemistry type definitions

In [2]:
pdb_path = Path('pdb_files')
pdb_path.mkdir(exist_ok=True)

coll_path = Path('Collections')
coll_path.mkdir(exist_ok=True)

lammps_path = Path('LAMMPS')
lammps_path.mkdir(exist_ok=True)

omm_path = Path('OpenMM')
omm_path.mkdir(exist_ok=True)

In [3]:
# defining reacting functional groups
reaction_pairs = {
    'NIPU' : ('cyclocarbonate', 'amine'),
    'urethane' : ('isocyanate', 'hydroxyl')
}
# chemistries = ('urethane', 'NIPU')
chemistries = [i for i in reaction_pairs.keys()]

# Collating urethanes into collections and generating Interchange files

In [5]:
from polysaccharide import OPENFF_DIR
from polysaccharide.charging.application import MolCharger

# specify forcefield
ff_name = 'openff-2.0.0.offxml'
# ff_name = 'openff_unconstrained-2.0.0.offxml'
ff_path = OPENFF_DIR / ff_name
forcefield = ForceField(ff_path)

# specify charging method
chg_method = 'Espaloma_AM1BCC'

In [ ]:
from tqdm.notebook import tqdm

success_ics = defaultdict(defaultdict)
failed_ics  = defaultdict(lambda : defaultdict(list))

for chemistry in chemistries:
    chem_path = coll_path / chemistry
    lmp_dir = lammps_path / chemistry
    lmp_dir.mkdir(exist_ok=True)

    mgr = PolymerManager(chem_path)
    for mol_name, polymer in ( progress := tqdm(mgr.polymers.items()) ):
        progress.set_postfix_str(f'{chemistry} : {mol_name}')
        try:
            chgr = MolCharger.subclass_registry[chg_method]()
            polymer.assert_charges_for(chgr, strict=True, return_cmol=False)

            sdf_path = polymer.structure_files_chgd[chg_method]
            cmol = polymer.charged_offmol_from_sdf(chg_method)
            offtop = Topology.from_molecules(cmol) # load topology from SDF file

            ic = forcefield.create_interchange(offtop, charge_from_molecules=[cmol])
            success_ics[chemistry][mol_name] = ic
            # ic.to_lammps(lmp_dir / f'{mol_name}.lammps')

        except AttributeError as a:
            print(a)

        except Exception as e:
            print(e)
            failed_ics[chemistry][e.__class__.__name__].append(mol_name)

In [ ]:
success_ics, failed_ics

# Running OpenMM simulations

## Defining utility functions

In [8]:
from openmm import XmlSerializer
from openmm import System, Context, State
from openmm import Integrator, Force
from openmm.app import Simulation
from openmm.unit import nanometer

from openff.interchange import Interchange
from openff.units import unit as offunit

from polysaccharide.simulation.ensemble import EnsembleSimulationFactory
from polysaccharide.simulation.records import SimulationParameters

DEFAULT_STATE_PARAMS : dict[str, bool] = {
    'getPositions'  : True,
    'getVelocities' : True,
    'getForces'     : True,
    'getEnergy'     : True,
    'getParameters' : True,
    'getParameterDerivatives' : False,
    'getIntegratorParameters' : False
}


def serialize_state_and_sys(sim : Simulation, out_dir : Path, out_name : str, state_params : dict[str, bool]=DEFAULT_STATE_PARAMS) -> None:
    '''For saving State and System info of a Simulation to disc'''
    sim_dict = {
        'system' : sim.system,
        'state' : sim.context.getState(**state_params)
    }
    
    for affix, save_data in sim_dict.items():
        save_path = out_dir / f'{out_name}_{affix}.xml'
        save_path.touch()

        with save_path.open('w') as file:
            file.write( XmlSerializer.serialize(save_data) )

def apply_state_to_context(state : State, context : Context) -> None:
    '''For applying saved State data to an existing OpenMM Simulation'''
    context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
    context.setPositions(state.getPositions())
    context.setVelocities(state.getVelocities())
    context.setTime(state.getTime())

    context.reinitialize(preserveState=True)    

def load_openmm_system(sys_path : Path, extra_forces : Optional[Union[Force, Iterable[Force]]]=None, sep_force_grps : bool=True, remove_constrs : bool=False) -> System:
    '''Load and configure a serialized OpenMM system, with optional additional parameters'''
    assert(sys_path.suffix == '.xml')
    with sys_path.open('r') as file:
        ommsys = XmlSerializer.deserialize(file.read())

    if extra_forces: # deliberately sparse to handle both Nonetype and empty list
        for force in extra_forces: 
            ommsys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(ommsys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(ommsys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            ommsys.removeConstraint(i)

    return ommsys

def create_simulation2(interchange : Interchange, integrator : Integrator, forces : Optional[Iterable[Force]]=None,
                        sep_force_grps : bool=True, remove_constrs : bool=True, combine_nonbonded_forces : bool=True) -> Simulation:
    '''Specifies configuration for an OpenMM Simulation - Interchange load alows many routes for creation'''
    openmm_sys = interchange.to_openmm(combine_nonbonded_forces=combine_nonbonded_forces) 
    openmm_top = interchange.topology.to_openmm()
    openmm_pos = interchange.positions.m_as(offunit.nanometer) * nanometer

    if forces: # deliberately sparse to handle both Nonetype and empty list
        for force in forces: 
            openmm_sys.addForce(force)

    if sep_force_grps:
        for i, force in enumerate(openmm_sys.getForces()):
            force.setForceGroup(i)

    if remove_constrs:
        for i in range(openmm_sys.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
            openmm_sys.removeConstraint(i)

    simulation = Simulation(openmm_top, openmm_sys, integrator)
    simulation.context.setPositions(openmm_pos)

    return simulation

def openmm_sim_from_files(top_file : Path, sys_file : Path, state_file : Path, sim_params : SimulationParameters, sep_force_grps : bool=True, remove_constrs : bool=False) -> Simulation:
    '''Load an OpenMM simulation object from serialized Topology, System, and State'''
    offmol = Molecule.from_file(top_file)
    offtop = Topology.from_molecules(offmol)
    ommtop = offtop.to_openmm()

    # define ensemble-specific forces and Integrator
    ens_fac = EnsembleSimulationFactory.subclass_registry[sim_params.ensemble.upper()]()
    integrator = ens_fac.integrator(sim_params)
    forces     = ens_fac.forces(    sim_params)

    # load and configure System
    ommsys = load_openmm_system(sys_file, extra_forces=forces, sep_force_grps=sep_force_grps, remove_constrs=remove_constrs)

    # putting it all together into a Simulation
    return Simulation(
        topology=ommtop,
        system=ommsys,
        integrator=integrator,
        state=state_file
    )

# load and apply State
# with state_file.open('r') as file:
#     ommstate = XmlSerializer.deserialize(file.read())
# apply_state_to_context(ommstate, sim.context)

## Serialize OpenMM simulations to file

In [9]:
# specifying simulation and ensemble parameters
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole
from shutil import copyfile
from copy import deepcopy

from polysaccharide import filetree
from polysaccharide.simulation.records import SimulationParameters
from polysaccharide.simulation.ensemble import EnsembleSimulationFactory


omm_dir = Path('OpenMM')
omm_dir.mkdir(exist_ok=True)

# selecting simulation parameters and ensemble
sp_path = Path('debug_sim_NVT.json')
sim_params = SimulationParameters.from_file(sp_path)
ens_fac = EnsembleSimulationFactory.subclass_registry[sim_params.ensemble.upper()]()

# looping over all urethanes
omm_sims = defaultdict(defaultdict)
for chemistry, ic_dict in success_ics.items():
    mgr = PolymerManager(coll_path / chemistry)
    chem_dir = omm_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    for mol_name, interchange in ic_dict.items():
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }

        # creating directories
        mol_dir = chem_dir / mol_name 
        mol_dir.mkdir(exist_ok=True)

        # creating simulation and associated files
        polymer = mgr.polymers[mol_name]
        sdf_path = polymer.structure_files_chgd[chg_method]

        # create and register simulation
        sim = ens_fac.create_simulation(interchange, sim_params)
        omm_sims[chemistry][mol_name] = sim

        # serialize Topology, System, and State for reloading
        sdf_out_path = mol_dir / f'{mol_name}_topology.sdf'
        copyfile(sdf_path, sdf_out_path)
        serialize_state_and_sys(sim, out_dir=mol_dir, out_name=mol_name)

## Loading simulations from file

In [36]:
# parameters
sep_force_grps : bool = True
remove_constrs : bool = False

# paths
# omm_dir = Path('OpenMM')
omm_dir = Path('OpenMM_unconstrained')
sp_path = Path('debug_sim_NVT.json')

# iterate over serialized directory tree and load
sim_params = SimulationParameters.from_file(sp_path)

omm_sims = defaultdict(defaultdict)
for subdir in omm_dir.iterdir():
    if subdir.is_dir():
        chemistry = subdir.name
        for mol_dir in subdir.iterdir():
            mol_name = mol_dir.name

            state_file = mol_dir / f'{mol_name}_state.xml'
            sys_file   = mol_dir / f'{mol_name}_system.xml'
            top_file   = mol_dir / f'{mol_name}_topology.sdf'

            omm_sims[chemistry][mol_name] = openmm_sim_from_files(top_file, sys_file, state_file, sim_params, sep_force_grps=sep_force_grps, remove_constrs=remove_constrs)

## Evaluating starting structure energies

In [37]:
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole

repl_table = {
    'PeriodicTorsion' : 'Torsion',
    'HarmonicAngle' : 'Angle',
    'HarmonicBond' : 'Bond'
}

NULL_ENERGY = 0.0*kilojoule_per_mole
PRECISION : int = 4

remove_constrs : bool = True

data_dicts = []
for chemistry, mol_dict in omm_sims.items():
    for mol_name, sim in mol_dict.items():
        # assign and initialize unique force groups for simulation
        for i, force in enumerate(sim.system.getForces()):
            force.setForceGroup(i)
            # print(force.getName(), force.getForceGroup())
        sim.context.reinitialize(preserveState=True) # need to reinitialize to get force labelling changes to "stick"

        # remove constraints
        if remove_constrs:
            for i in range(sim.system.getNumConstraints())[::-1]: # need to remove in reverse order to avoid having prior constraints "fall back down"
                sim.system.removeConstraint(i)
            sim.context.reinitialize(preserveState=True)

        # extract total and component energies from OpenMM force groups
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }
        omm_energies = {}

        overall_state = sim.context.getState(getEnergy=True) # get total potential energy
        PE = overall_state.getPotentialEnergy()
        omm_energies['Potential'] = PE

        KE = overall_state.getKineticEnergy()
        omm_energies['Kinetic'] = KE
        assert(KE == NULL_ENERGY)

        for i, force in enumerate(sim.system.getForces()):
            state = sim.context.getState(getEnergy=True, groups={i})
            force_name = force.getName().removesuffix('Force')
            omm_energies[force_name] = state.getPotentialEnergy()

        # reformat
        omm_energies_kcal = {}
        for contrib, energy_kj in omm_energies.items():
            contrib_name = repl_table.get(contrib, contrib) # check if replacement is defined, return original name otherwise
            energy_kcal = energy_kj.in_units_of(kilocalorie_per_mole)
            omm_energies_kcal[f'{contrib_name} ({energy_kcal.unit.get_symbol()})'] = round(energy_kcal._value, PRECISION)

        # compiling data
        data_dict = {**data_dict, **omm_energies_kcal}
        data_dicts.append(data_dict)

df = pd.DataFrame.from_records(data_dicts)
df.sort_values('Molecule', inplace=True)
df.to_csv(omm_dir / f'{omm_dir.name}_PEs.csv', index=False)

## Comparing energies from constrained and unconstrained FFs

In [56]:
e_tables = {
    platform : pd.read_csv(Path(platform) / f'{platform}_PEs.csv', index_col=(0, 1))
        for platform in ('LAMMPS', 'OpenMM', 'OpenMM_unconstrained')
}

In [ ]:
cols = [
    'Nonbonded (kcal/mol)',
    'Bond (kcal/mol)',
    'Angle (kcal/mol)',
    'Torsion (kcal/mol)',
    'Potential (kcal/mol)',
    'Kinetic (kcal/mol)',
]

for src, tab in e_tables.items():
    print(src)
    display(tab[cols])

In [63]:
diff = e_tables['OpenMM_unconstrained'] - e_tables['LAMMPS']
diff

Angle (kcal/mol)  Bond (kcal/mol)  \
Chemistry Molecule                                         
NIPU      NIPU_11              0.000130        -0.000117   
          NIPU_2              -0.000078        -0.002156   
urethane  urethane_0          -0.000203         0.001079   
          urethane_1          -0.000086         0.000364   
          urethane_10          0.000075        -0.000266   
          urethane_14         -0.000270         0.000227   
          urethane_15         -0.000069        -0.000166   
          urethane_17         -0.000282        -0.000325   
          urethane_2          -0.000142         0.000002   
          urethane_22          0.000240        -0.001099   
          urethane_24         -0.000482         0.001298   
          urethane_29          0.000053         0.000518   
          urethane_35         -0.000113         0.001283   
          urethane_41         -0.000477        -0.001436   
          urethane_42         -0.000283        -0.002401   
          urethane_44         -0.000056         0.000570   
          urethane_45         -0.000199         0.002569   
          urethane_47         -0.000224         0.000452   
          urethane_48          0.001845        -0.000906   
          urethane_51         -0.000139        -0.000076   
          urethane_52         -0.000077         0.000227   
          urethane_6           0.000366        -0.001365   
          urethane_8          -0.000155         0.000434   

                       Impropers (kcal/mol)  Kinetic (kcal/mol)  \
Chemistry Molecule                                                
NIPU      NIPU_11                       NaN                 0.0   
          NIPU_2                        NaN                 0.0   
urethane  urethane_0                    NaN                 0.0   
          urethane_1                    NaN                 0.0   
          urethane_10                   NaN                 0.0   
          urethane_14                   NaN                 0.0   
          urethane_15                   NaN                 0.0   
          urethane_17                   NaN                 0.0   
          urethane_2                    NaN                 0.0   
          urethane_22                   NaN                 0.0   
          urethane_24                   NaN                 0.0   
          urethane_29                   NaN                 0.0   
          urethane_35                   NaN                 0.0   
          urethane_41                   NaN                 0.0   
          urethane_42                   NaN                 0.0   
          urethane_44                   NaN                 0.0   
          urethane_45                   NaN                 0.0   
          urethane_47                   NaN                 0.0   
          urethane_48                   NaN                 0.0   
          urethane_51                   NaN                 0.0   
          urethane_52                   NaN                 0.0   
          urethane_6                    NaN                 0.0   
          urethane_8                    NaN                 0.0   

                       Nonbonded (kcal/mol)  Potential (kcal/mol)  \
Chemistry Molecule                                                  
NIPU      NIPU_11              7.583564e+02          7.583570e+02   
          NIPU_2               4.629803e+02          4.629781e+02   
urethane  urethane_0          -2.504192e+03         -2.504191e+03   
          urethane_1          -7.171534e+02         -7.171529e+02   
          urethane_10         -3.877483e+02         -3.877482e+02   
          urethane_14         -3.044780e+02         -3.044781e+02   
          urethane_15          4.761875e+03          4.761875e+03   
          urethane_17         -2.737855e+03         -2.737855e+03   
          urethane_2           1.476992e+06          1.476992e+06   
          urethane_22         -9.113365e+05         -9.113365e+05   
          urethane

In [64]:
diff['Torsion (kcal/mol)'] - e_tables['LAMMPS']['Impropers (kcal/mol)']

Chemistry  Molecule   
NIPU       NIPU_11        0.000005
           NIPU_2         0.000094
urethane   urethane_0     0.000026
           urethane_1     0.000055
           urethane_10    0.000057
           urethane_14    0.000177
           urethane_15    0.000021
           urethane_17    0.000057
           urethane_2     0.000015
           urethane_22    0.000145
           urethane_24   -0.000002
           urethane_29    0.000020
           urethane_35    0.000014
           urethane_41   -0.000033
           urethane_42    0.000018
           urethane_44    0.000096
           urethane_45    0.000103
           urethane_47    0.000057
           urethane_48    0.000185
           urethane_51    0.000104
           urethane_52    0.000026
           urethane_6     0.000066
           urethane_8     0.000041
dtype: float64

In [ ]:
df_unconstr.loc[[('urethane', 'urethane_0'), ('urethane', 'urethane_1'), ('urethane', 'urethane_41')]]

In [ ]:
nonbond = sim.system.getForce(0)
nonbond.getCutoffDistance()

In [ ]:
nonbond.getName()

In [ ]:
data_dicts = []
for chemistry, mol_dict in omm_sims.items():
    print(chemistry)
    for mol_name, sim in mol_dict.items():
        print(mol_name)
        nonbond = sim.system.getForce(0)
        assert(nonbond.getName() == 'NonbondedForce')
        print(nonbond.getCutoffDistance())

In [ ]:
import matplotlib.pyplot as plt

bond_diffs = e_diff['HarmonicBond (kcal/mol)'].to_numpy()
plt.hist(bond_diffs, bins=20)
plt.xlim(bond_diffs.min(), bond_diffs.max())
plt.xlabel(f'{general.GREEK_UPPER["delta"]}E_bond const vs unconst')
plt.ylabel('Count')

## Probing NonbondedForce

In [ ]:
nonbond = sim.system.getForce(0)

In [ ]:
SIG = general.GREEK_LOWER['sigma']
EPS = general.GREEK_LOWER['epsilon']

records = []
for atom in sim.topology.atoms():
    charge, sigma, epsilon = nonbond.getParticleParameters(atom.index)
    records.append({
        'Atom name' : atom.name,
        'Element' : atom.element.symbol,
        'Partial charge' : charge,
        f'LJ_{SIG}' : sigma,
        f'LJ_{EPS}' : epsilon
    })

lj_dframe = pd.DataFrame.from_records(records)
lj_dframe

In [ ]:
targ_elem = 'C'

targ_atoms = [
    (atom.index, atom)
        for atom in sim.topology.atoms()
            if atom.element.symbol == targ_elem
]

## Testing effect of removing constraints on energies

In [ ]:
ic = ic_dict['urethane_44']
integrator = ens_fac.integrator(sim_params)
forces = ens_fac.forces(sim_params)

In [ ]:
simm = create_simulation2(ic, integrator, forces=forces, sep_force_grps=True, remove_constrs=True, combine_nonbonded_forces=True)

In [ ]:
simm.system.getNumConstraints()

In [ ]:
overall_state = simm.context.getState(getEnergy=True) # get total potential energy
PE = overall_state.getPotentialEnergy()
omm_energies['Total'] = PE

KE = overall_state.getKineticEnergy()
assert(KE == NULL_ENERGY)

for i, force in enumerate(simm.system.getForces()):
    state = simm.context.getState(getEnergy=True, groups={i})
    force_name = force.getName().removesuffix('Force')
    omm_energies[force_name] = state.getPotentialEnergy()

# reformat
omm_energies_kcal = {}
for contrib, energy_kj in omm_energies.items():
    energy_kcal = energy_kj.in_units_of(kilocalorie_per_mole)
    omm_energies_kcal[f'{contrib} ({energy_kcal.unit.get_symbol()})'] = round(energy_kcal._value, PRECISION)


In [ ]:
omm_energies_kcal

In [ ]:
for i in range(sim.system.getNumConstraints())[::-1]:
    print(sim.system.getConstraintParameters(i))
    sim.system.removeConstraint(i)

In [ ]:
list(range(5)[::-1])

In [ ]:
sim.system.getNumConstraints()

In [ ]:
sim.system.getConstraintParameters(359)

## Comparing ParmEd energy decomposition to native OpenMM force-group-based decomposition

In [ ]:
import parmed
from openmm.openmm import Force

NULL_ENERGY = 0.0*kilojoule_per_mole

sim = omm_sims['urethane']['urethane_41']
# assign and initialize unique force groups for simulation
for i, force in enumerate(sim.system.getForces()):
    force.setForceGroup(i)
    # print(force.getName(), force.getForceGroup())
sim.context.reinitialize(preserveState=True) # need to reinitialize to get force labelling changes to "stick"

# energies from OpenMM force groups
print('\nOpenMM:')
print('='*30)
omm_energies = {}

## extract total energies for state
overall_state = sim.context.getState(getEnergy=True) # get total potential energy
PE = overall_state.getPotentialEnergy()
omm_energies['Total Potential Energy'] = PE

KE = overall_state.getKineticEnergy()
assert(KE == NULL_ENERGY)

for i, force in enumerate(sim.system.getForces()):
    state = sim.context.getState(getEnergy=True, groups={i})
    force_name = force.getName().removesuffix('Force')
    pe = state.getPotentialEnergy()

    omm_energies[force_name] = pe
    print(f'{force_name} : {pe}')

## converting name to match with ParmEd for comparison
namemap = {
    'Nonbonded' : 'bond',
    'PeriodicTorsion' : 'angle',
    'HarmonicAngle' : 'dihedral',
    'HarmonicBond' : 'urey_bradley',
    'Total Potential Energy' : 'total'
}
compat_omm_energies = {
    namemap[contrib] : energy
        for contrib, energy in omm_energies.items()
}

total = sum(omm_energies.values(), start=NULL_ENERGY) # need "seed" to have Quantity datatype to sum
print(f'{general.GREEK_UPPER["delta"]}E_contrib: ', PE - total)

# ParmEd energy decomposition
print('\nParmEd:')
print('='*30)
parm_energies = {}
parm_struct = parmed.openmm.load_topology(sim.topology, sim.system)
for contrib, energy_val in parmed.openmm.energy_decomposition(parm_struct, sim.context).items():
    parm_energies[contrib] = energy = energy_val*kilocalorie_per_mole # assign proper units
    print(contrib, energy.in_units_of(kilojoule_per_mole))

## Minimizing and running single integration step, then evaluating energies from reporter

In [ ]:
import re
from openmm.unit import kilojoule_per_mole, kilocalorie_per_mole

from polysaccharide.simulation import preparation
from polysaccharide.simulation.records import SimulationParameters
from polysaccharide.simulation.ensemble import EnsembleSimulationFactory

STRIP_BEFORE_PARENS = re.compile(r'(.*?)(?=\s*\(.*\))')
PRECISION = 3 # number of decimals to round reported energies to

omm_dir = Path('OpenMM_no_sim')
omm_dir.mkdir(exist_ok=True)


# selecting simulation parameters and ensemble
sp_path = Path('debug_sim_NVT.json')
sim_params = SimulationParameters.from_file(sp_path)
ens_fac = EnsembleSimulationFactory.subclass_registry[sim_params.ensemble.upper()]()

data_by_mol = []
omm_sims = defaultdict(defaultdict)
# looping over all urethanes
for chemistry, ic_dict in success_ics.items():
    chem_dir = omm_dir / chemistry
    chem_dir.mkdir(exist_ok=True)

    for mol_name, interchange in ic_dict.items():
        data_dict = {
            'Chemistry' : chemistry,
            'Molecule'  : mol_name
        }

        # creating directories
        mol_dir = chem_dir / mol_name 
        mol_dir.mkdir(exist_ok=True)

        sim_file_dir = mol_dir / f'{mol_name}_sim'
        sim_file_dir.mkdir(exist_ok=True, parents=True)

        # creating simulation and associated files
        sim = ens_fac.create_simulation(interchange, sim_params)
        sim_paths = preparation.prepare_simulation_paths(output_folder=sim_file_dir, output_name=mol_name, sim_params=sim_params)
        reporters = preparation.prepare_simulation_reporters(sim_paths, sim_params)
        preparation.config_simulation(sim, reporters, checkpoint_path=sim_paths.checkpoint)

        # energy min and single-step integration
        sim.minimizeEnergy()
        sim.step(1)

        # extracting energies

        state_data = pd.read_csv(sim_paths.state_data)
        energies = {}
        for key in ('Potential Energy (kJ/mole)', 'Kinetic Energy (kJ/mole)'):
            tag = re.search(STRIP_BEFORE_PARENS, key).group(0)
            E_kj_val = state_data[key][0]
            E_kj = E_kj_val * kilojoule_per_mole
            E_kcal = E_kj.in_units_of(kilocalorie_per_mole)

            for energy in (E_kj, E_kcal):
                energies[f'{tag} ({energy.unit.get_symbol()})'] = energy._value
        
        data_dict.update(**energies)
        data_by_mol.append(data_dict)

# collate energies into DataFrame        
df = pd.DataFrame.from_records(data_by_mol)
df = df.sort_values('Molecule')

# round energy values down to desired precision
round_fn = lambda x : round(x, PRECISION)

for col_name, col in df.items():
    try:
        df[col_name] = col.apply(round_fn) # attempt to round column and replace with rounded values
    except TypeError:
        pass

# save energies to file
energy_file = omm_dir / 'energies_1_step.csv'
df.to_csv(energy_file, index=False)

# Inspecting which molecules fail and succeed and why

In [ ]:
import re
regex = re.compile(r'\A(\w+_\d+)')

diff = {}
for chemistry, succ_names in success_ics.items():
    chem_dir = coll_path / chemistry
    for sdf_path in chem_dir.glob('**/*.sdf'):
        mol_name = re.search(regex, sdf_path.stem).group()
        if mol_name not in succ_names:
            diff.append(mol_name)

## Checking for successful residue covers of newly-generated PDB Topologies

In [ ]:
chemistry = 'urethane'

mgr = PolymerManager(coll_path / chemistry)
mol_names = failed_interchanges[chemistry]['UnmatchedAtomsError']

offmols = {
    mol_name : mgr.polymers[mol_name].offmol_matched(strict=False)
        for mol_name in mol_names
}

In [ ]:
sizes = {
    mol_name : offmol.n_atoms
        for mol_name, offmol in sorted(offmols.items(), key=lambda x : x[1].n_atoms)
}

In [ ]:
sizes

In [ ]:

pdir = mgr.polymers['urethane_6']
# pdir = mgr.polymers['NIPU_8']
# pdir.offmol_matched(strict=True)

for atom in offmol.atoms:
    if not atom.metadata['already_matched']:
        print(atom.metadata)

In [ ]:
mgr = PolymerManager(coll_path / 'NIPU')

offmols = {}
unmatched = []
for mol_name, polymer in mgr.polymers.items():
    try:
        offmols[mol_name] = polymer.offmol
    except:
        unmatched.append(mol_name)

In [ ]:
for mol_name in unmatched:
    print(mol_name)
    polymer = mgr.polymers[mol_name]
    offmol = polymer.offmol_matched(strict=False)
    
    for atom in offmol.atoms:
        if not atom.metadata['already_matched']:
            print('\t', atom.metadata)

# Experimenting with SDF files

In [ ]:
benz = Chem.MolFromSmiles('C1ccccC=1')
benz = Chem.AddHs(benz)
benz.SetDoubleProp('stuff', 3.14)
benz

In [ ]:
block2k = Chem.MolToMolFile(benz, 'test_2k.sdf')
block3k = Chem.MolToV3KMolFile(benz, 'test_3k.sdf')

In [ ]:
block2kforce = Chem.MolToMolFile(benz, 'test_2k_force.sdf', forceV3000=True)

In [ ]:
with Chem.SDWriter('test_sdw.sdf') as sdwriter:
    sdwriter.SetForceV3000(True)
    print(sdwriter.GetForceV3000())

    sdwriter.write(benz)

In [ ]:
with Chem.SDMolSupplier('sdf_testing/test_off_rd.sdf', sanitize=False) as suppl:
    mols = [mol for mol in suppl]

targ = mols[0]
targ

In [ ]:
omol = Molecule.from_rdkit(benz)
omol.generate_conformers(n_conformers=1)
omol.visualize(backend='nglview')

In [ ]:
from polysaccharide import filetree
from polysaccharide import TOOLKITS


p = Path('sdf_testing/test_off_rd.sdf')
tkwrap = TOOLKITS['The RDKit']

omol.properties['series'] = (1,2,3)
omol.to_file(
    general.asstrpath(p),
    file_format=filetree.dotless(p.suffix),
    toolkit_registry=tkwrap
)

In [ ]:
omol_load = Molecule.from_file(
    general.asstrpath(p),
    file_format=filetree.dotless(p.suffix),
    toolkit_registry=tkwrap
)

In [ ]:
omol_load.properties

In [ ]:
help(tkwrap.to_file)

In [ ]:
help(omol.to_file)